## 使用ID3决策树算法预测销量高低

![图一：销量数据实例](http://aliyuntianchipublic.cn-hangzhou.oss-pub.aliyun-inc.com/public/files/image/null/1541384589972_6zXK9APiUJ.jpg)

In [1]:
# -*- coding: utf-8 -*-
#使用ID3决策树算法预测销量高低

import pandas as pd
import numpy as np

#参数初始化
filename = 'sales_data.csv'
#导入数据。 index_col参数指定了索引列
data = pd.read_csv(filename, index_col = u'序号')

print("data.columns:\n{}\n", format(data.columns))
print("data.index:\n{}\n", format(data.index))

#数据都是类别标签， 要将他转化为数据
#用 1来表示好，是，高，这三个属性
data[data == u'好'] = 1   #如果某个值是“好”，那就给他重新赋值为1.
data[data == u'是'] = 1
data[data == u'高'] = 1
#用 -1 来表示坏，否，低，这三个属性
data[data != 1] = -1   #上面没有重新赋值的，都给赋值为 -1

# 把数据分别付给两个dataframe变量，以便于后面作为决策树算法的输入
#iloc[:,:3]表示第0，1，2列数据，也就是“天气”、“是否周末”、“是否促销” 这三列。python中的范围区间默认是前闭后开
#iloc[:,3]表示第3列数据，也就是“销量”这列
x = data.iloc[:,:3].as_matrix().astype(int)
y = data.iloc[:,3].as_matrix().astype(int)

#as_matrix()把数据转为矩阵，astype(int)把数据转为int型

#下面重点来了：建立决策树
#先把模块导入进来
from sklearn.tree import DecisionTreeClassifier as dtc
#criterion参数设置为信息熵模式
dtcx = dtc(criterion = 'entropy')
#核心的代码其实就是下面这发，把数据送入算法函数进行训练
dtcx.fit(x, y)
print("fit done!")
pr = dtcx.predict(np.array([[1,1,1],[-1,1,-1]]))
print("predict done:", format(pr))

#下面是可视化的部分，把刚才训练好的决策树模型可视化输出。把他当成print()就行
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO
x = pd.DataFrame(x)
with open("tree.dot",'w') as f:
    f = export_graphviz(dtcx,feature_names=x.columns,out_file=f)


#下面的代码是将生成的png图打印出来。因为数据列表里不让传图，所以代码中引用的是我本地的图片路径。
'''
from PIL import Image
import matplotlib.pyplot as plt

img=Image.open('D:/python_work/tree.png')
plt.rcParams['figure.dpi'] = 200
plt.imshow(img)
'''

data.columns:
{}
 Index(['天气', '是否周末', '是否有促销', '销量'], dtype='object')
data.index:
{}
 Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
            18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34],
           dtype='int64', name='序号')


F:\anaconda\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
F:\anaconda\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


fit done!
predict done: [ 1 -1]


"\nfrom PIL import Image\nimport matplotlib.pyplot as plt\n\nimg=Image.open('D:/python_work/tree.png')\nplt.rcParams['figure.dpi'] = 200\nplt.imshow(img)\n"



##### 树图无法直接作为结果输出出来，只能贴出来png.   


![通过Graphviz绘制的决策树：](http://aliyuntianchipublic.cn-hangzhou.oss-pub.aliyun-inc.com/public/files/image/null/1541393801453_IJAk9Otpbw.jpg)

  
  
#### 自问自答

1. ID3决策树算法中涉及到了信息熵、信息增益等概念，如何理解？  

> **$$I = - P(x)\log_2 {P(x)}$$**    这个公式可以分为2部分，一个是`P(x)`，这是x发生的概率；另外一部分是$-\log_2 {P(x)}$，它表示需要几个bit来度量P(x)这个信息量；加了负号是为了将bit数取正值，因为概率型数值大部分都小于1，它的对数都是负值；再加个负号就负负得正啦。所以这个负号主要是为了方便度量信息量而加的。  

> $-\log_2 {P(x)}$ 这部分表达式中，为什么说它表示需要几个bit来度量P(x)呢？ 这个就需要看下bit的定义，我百度了一下，对这个概念重新认识了一下：  

> 比特（bit）是表示信息的最小单位，是二进制数的一位包含的信息或 **2个选项中特别指定1个的需要信息量**。一般来说，n比特的信息量可以表现出2的n次方种选择。  

> 信息增益越大，不确定性越小。

2. 对于最后输出的树形图，如何对它进行解读？  

> 